In [2]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
# %matplotlib inline
import pickle as pickle
# 이거 데이터프레임 숫자 표시 개수 정해주는 거
pd.options.display.float_format = '{:.4f}'.format
import warnings
warnings.filterwarnings('ignore')
import matplotlib.gridspec as gridspec

In [3]:
dev_loss_st = pd.read_csv('/opt/ml/code/prediction/0_dev_loss_st.csv')
train = pd.read_csv('/opt/ml/dataset/train/train.csv')
test = pd.read_csv('/opt/ml/code/prediction/0_test_st.csv')

In [3]:
test=test.drop(columns='gold_label')

In [4]:
test['probs'].apply(lambda x: str(np.append([1],np.zeros_like(eval(test.iloc[0,2]))[:-1])))

0       [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...
1       [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...
2       [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...
3       [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...
4       [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...
                              ...                        
7760    [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...
7761    [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...
7762    [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...
7763    [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...
7764    [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...
Name: probs, Length: 7765, dtype: object

In [5]:
test['probs'].apply(lambda x: str([0]*30))

0       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
2       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
3       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
4       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
                              ...                        
7760    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
7761    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
7762    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
7763    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
7764    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: probs, Length: 7765, dtype: object

In [6]:
# test.to_csv('/opt/ml/code/prediction/24 new_meta_binary2.csv',index=False)

In [7]:
num_label = []
label = dev_loss_st['gold_label'].unique()
with open('/opt/ml/code/dict_label_to_num.pkl', 'rb') as f:
    dict_label_to_num = pickle.load(f)
    for v in label:
        num_label.append(dict_label_to_num[v])

In [8]:
pd.DataFrame([dict_label_to_num])

,no_relation,org:top_members/employees,org:members,org:product,per:title,org:alternate_names,per:employee_of,org:place_of_headquarters,per:product,org:number_of_employees/members,...,org:member_of,per:parents,org:dissolved,per:schools_attended,per:date_of_death,per:date_of_birth,per:place_of_birth,per:place_of_death,org:founded_by,per:religion
0,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29


In [23]:
def train_pred(train):
    subjt=train['subject_entity'].apply(lambda x: pd.Series(eval(x))).add_prefix('subj_')
    objt=train['object_entity'].apply(lambda x: pd.Series(eval(x))).add_prefix('obj_')
    full_train = pd.concat([train[['id','sentence','source','label']],subjt,objt],axis=1)
    del subjt,objt
    return full_train

In [24]:
full_train=train_pred(train)

In [25]:
train.head(5)

,id,sentence,subject_entity,object_entity,label,source
0,0,〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey R...,"{'word': '비틀즈', 'start_idx': 24, 'end_idx': 26...","{'word': '조지 해리슨', 'start_idx': 13, 'end_idx':...",no_relation,wikipedia
1,1,호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으...,"{'word': '민주평화당', 'start_idx': 19, 'end_idx': ...","{'word': '대안신당', 'start_idx': 14, 'end_idx': 1...",no_relation,wikitree
2,2,K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹으로부터...,"{'word': '광주FC', 'start_idx': 21, 'end_idx': 2...","{'word': '한국프로축구연맹', 'start_idx': 34, 'end_idx...",org:member_of,wikitree
3,3,균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪...,"{'word': '아성다이소', 'start_idx': 13, 'end_idx': ...","{'word': '박정부', 'start_idx': 22, 'end_idx': 24...",org:top_members/employees,wikitree
4,4,1967년 프로 야구 드래프트 1순위로 요미우리 자이언츠에게 입단하면서 등번호는 8...,"{'word': '요미우리 자이언츠', 'start_idx': 22, 'end_id...","{'word': '1967', 'start_idx': 0, 'end_idx': 3,...",no_relation,wikipedia


In [26]:
full_train.head(5)

,id,sentence,source,label,subj_word,subj_start_idx,subj_end_idx,subj_type,obj_word,obj_start_idx,obj_end_idx,obj_type
0,0,〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey R...,wikipedia,no_relation,비틀즈,24,26,ORG,조지 해리슨,13,18,PER
1,1,호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으...,wikitree,no_relation,민주평화당,19,23,ORG,대안신당,14,17,ORG
2,2,K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹으로부터...,wikitree,org:member_of,광주FC,21,24,ORG,한국프로축구연맹,34,41,ORG
3,3,균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪...,wikitree,org:top_members/employees,아성다이소,13,17,ORG,박정부,22,24,PER
4,4,1967년 프로 야구 드래프트 1순위로 요미우리 자이언츠에게 입단하면서 등번호는 8...,wikipedia,no_relation,요미우리 자이언츠,22,30,ORG,1967,0,3,DAT


In [27]:
full_train[full_train['subj_type']=='PER']['obj_type'].unique()

array(['DAT', 'ORG', 'LOC', 'POH', 'PER', 'NOH'], dtype=object)

In [28]:
full_train[full_train['subj_type']=='ORG']['obj_type'].unique()

array(['PER', 'ORG', 'DAT', 'LOC', 'POH', 'NOH'], dtype=object)

In [30]:
full_train[(full_train['subj_type']=='PER')&(full_train['subj_type']=='PER')]['label'].unique()

array(['per:date_of_birth', 'per:employee_of', 'per:origin', 'per:title',
       'no_relation', 'per:schools_attended', 'per:colleagues',
       'per:alternate_names', 'per:spouse', 'per:children',
       'per:place_of_birth', 'per:parents', 'per:religion',
       'per:date_of_death', 'per:place_of_residence', 'per:other_family',
       'per:siblings', 'per:product', 'per:place_of_death', 'org:members',
       'org:top_members/employees', 'org:founded_by',
       'org:place_of_headquarters'], dtype=object)

In [33]:
full_train['bi_label'] = full_train['label'].apply(lambda x: x=='no_relation' if x, 'relation'))

AttributeError: 'str' object has no attribute 'where'

In [ ]:
multi -> per : per / label 5개 아니면 no relation

In [4]:
a=train[train['subject_entity'].apply(lambda x: eval(x)['type']=='PER')]

In [7]:
b = train[train['object_entity'].apply(lambda x: eval(x)['type']=='PER')]

In [12]:
c = pd.merge(a,b,how='inner')

In [14]:
c

,id,sentence,subject_entity,object_entity,label,source
0,20,1971년 대선을 앞두고 김종필은 1971년 선거에서 박정희 당선을 위해 무려 60...,"{'word': '김종필', 'start_idx': 14, 'end_idx': 16...","{'word': '박정희', 'start_idx': 30, 'end_idx': 32...",per:colleagues,wikipedia
1,21,2010년에는 아시아 가수 최초로 마이클 잭슨의 곡을 리메이크하였는데 당시 마이클 ...,"{'word': '스티브 바라캇', 'start_idx': 67, 'end_idx'...","{'word': 'Steve Barakatt', 'start_idx': 75, 'e...",per:alternate_names,wikipedia
2,22,"박흥식은 첫 부인과의 사이에 장녀 박병숙을 두었고, 두 번째 부인은 경희대학교 교수...","{'word': '박흥식', 'start_idx': 0, 'end_idx': 2, ...","{'word': '한인하', 'start_idx': 57, 'end_idx': 59...",per:spouse,wikipedia
3,25,"SK는 또 ""해당 방송에서 언급한 지난해 12월5일 가로세로연구소 유튜브 방송 또한...","{'word': '노소영', 'start_idx': 110, 'end_idx': 1...","{'word': '최태원', 'start_idx': 67, 'end_idx': 69...",per:spouse,wikitree
4,27,"1824년에 있었던 첫 졸업식에는 대통령 제임스 먼로와 존 C. 칼훈, 헨리 클레이...","{'word': '존 C. 칼훈', 'start_idx': 31, 'end_idx'...","{'word': '제임스 먼로', 'start_idx': 23, 'end_idx':...",no_relation,wikipedia
...,...,...,...,...,...,...
5004,32438,"간 나오토는 총리 취임에 즈음해 당 간사장에 에다노 유키오, 내각관방장관에 센고쿠 ...","{'word': '에다노 유키오', 'start_idx': 25, 'end_idx'...","{'word': '센고쿠 요시토', 'start_idx': 42, 'end_idx'...",no_relation,wikipedia
5005,32451,"반면 정재학의 3남인 정종원은 제5대 조흥은행장, 손자인 정운용은 제2대 조흥은행장...","{'word': '정종원', 'start_idx': 12, 'end_idx': 14...","{'word': '정재학', 'start_idx': 3, 'end_idx': 5, ...",per:parents,wikipedia
5006,32454,신라에서는 중기까지도 지배계층의 가계계승에 있어서 모계적인 경향이 그대로 존속되었고...,"{'word': '선덕여왕', 'start_idx': 101, 'end_idx': ...","{'word': '진덕여왕', 'start_idx': 95, 'end_idx': 9...",no_relation,wikipedia
5007,32463,"그의 외삼촌은 루이 13세로, 남편인 루이 14세는 외사촌 오빠가 된다.","{'word': '루이 14세', 'start_idx': 21, 'end_idx':...","{'word': '루이 13세', 'start_idx': 8, 'end_idx': ...",per:other_family,wikipedia


In [15]:
c[c['object_entity'].apply(lambda x: eval(x)['type']=='PER')]

,id,sentence,subject_entity,object_entity,label,source
0,20,1971년 대선을 앞두고 김종필은 1971년 선거에서 박정희 당선을 위해 무려 60...,"{'word': '김종필', 'start_idx': 14, 'end_idx': 16...","{'word': '박정희', 'start_idx': 30, 'end_idx': 32...",per:colleagues,wikipedia
1,21,2010년에는 아시아 가수 최초로 마이클 잭슨의 곡을 리메이크하였는데 당시 마이클 ...,"{'word': '스티브 바라캇', 'start_idx': 67, 'end_idx'...","{'word': 'Steve Barakatt', 'start_idx': 75, 'e...",per:alternate_names,wikipedia
2,22,"박흥식은 첫 부인과의 사이에 장녀 박병숙을 두었고, 두 번째 부인은 경희대학교 교수...","{'word': '박흥식', 'start_idx': 0, 'end_idx': 2, ...","{'word': '한인하', 'start_idx': 57, 'end_idx': 59...",per:spouse,wikipedia
3,25,"SK는 또 ""해당 방송에서 언급한 지난해 12월5일 가로세로연구소 유튜브 방송 또한...","{'word': '노소영', 'start_idx': 110, 'end_idx': 1...","{'word': '최태원', 'start_idx': 67, 'end_idx': 69...",per:spouse,wikitree
4,27,"1824년에 있었던 첫 졸업식에는 대통령 제임스 먼로와 존 C. 칼훈, 헨리 클레이...","{'word': '존 C. 칼훈', 'start_idx': 31, 'end_idx'...","{'word': '제임스 먼로', 'start_idx': 23, 'end_idx':...",no_relation,wikipedia
...,...,...,...,...,...,...
5004,32438,"간 나오토는 총리 취임에 즈음해 당 간사장에 에다노 유키오, 내각관방장관에 센고쿠 ...","{'word': '에다노 유키오', 'start_idx': 25, 'end_idx'...","{'word': '센고쿠 요시토', 'start_idx': 42, 'end_idx'...",no_relation,wikipedia
5005,32451,"반면 정재학의 3남인 정종원은 제5대 조흥은행장, 손자인 정운용은 제2대 조흥은행장...","{'word': '정종원', 'start_idx': 12, 'end_idx': 14...","{'word': '정재학', 'start_idx': 3, 'end_idx': 5, ...",per:parents,wikipedia
5006,32454,신라에서는 중기까지도 지배계층의 가계계승에 있어서 모계적인 경향이 그대로 존속되었고...,"{'word': '선덕여왕', 'start_idx': 101, 'end_idx': ...","{'word': '진덕여왕', 'start_idx': 95, 'end_idx': 9...",no_relation,wikipedia
5007,32463,"그의 외삼촌은 루이 13세로, 남편인 루이 14세는 외사촌 오빠가 된다.","{'word': '루이 14세', 'start_idx': 21, 'end_idx':...","{'word': '루이 13세', 'start_idx': 8, 'end_idx': ...",per:other_family,wikipedia


In [16]:
c

,id,sentence,subject_entity,object_entity,label,source
0,20,1971년 대선을 앞두고 김종필은 1971년 선거에서 박정희 당선을 위해 무려 60...,"{'word': '김종필', 'start_idx': 14, 'end_idx': 16...","{'word': '박정희', 'start_idx': 30, 'end_idx': 32...",per:colleagues,wikipedia
1,21,2010년에는 아시아 가수 최초로 마이클 잭슨의 곡을 리메이크하였는데 당시 마이클 ...,"{'word': '스티브 바라캇', 'start_idx': 67, 'end_idx'...","{'word': 'Steve Barakatt', 'start_idx': 75, 'e...",per:alternate_names,wikipedia
2,22,"박흥식은 첫 부인과의 사이에 장녀 박병숙을 두었고, 두 번째 부인은 경희대학교 교수...","{'word': '박흥식', 'start_idx': 0, 'end_idx': 2, ...","{'word': '한인하', 'start_idx': 57, 'end_idx': 59...",per:spouse,wikipedia
3,25,"SK는 또 ""해당 방송에서 언급한 지난해 12월5일 가로세로연구소 유튜브 방송 또한...","{'word': '노소영', 'start_idx': 110, 'end_idx': 1...","{'word': '최태원', 'start_idx': 67, 'end_idx': 69...",per:spouse,wikitree
4,27,"1824년에 있었던 첫 졸업식에는 대통령 제임스 먼로와 존 C. 칼훈, 헨리 클레이...","{'word': '존 C. 칼훈', 'start_idx': 31, 'end_idx'...","{'word': '제임스 먼로', 'start_idx': 23, 'end_idx':...",no_relation,wikipedia
...,...,...,...,...,...,...
5004,32438,"간 나오토는 총리 취임에 즈음해 당 간사장에 에다노 유키오, 내각관방장관에 센고쿠 ...","{'word': '에다노 유키오', 'start_idx': 25, 'end_idx'...","{'word': '센고쿠 요시토', 'start_idx': 42, 'end_idx'...",no_relation,wikipedia
5005,32451,"반면 정재학의 3남인 정종원은 제5대 조흥은행장, 손자인 정운용은 제2대 조흥은행장...","{'word': '정종원', 'start_idx': 12, 'end_idx': 14...","{'word': '정재학', 'start_idx': 3, 'end_idx': 5, ...",per:parents,wikipedia
5006,32454,신라에서는 중기까지도 지배계층의 가계계승에 있어서 모계적인 경향이 그대로 존속되었고...,"{'word': '선덕여왕', 'start_idx': 101, 'end_idx': ...","{'word': '진덕여왕', 'start_idx': 95, 'end_idx': 9...",no_relation,wikipedia
5007,32463,"그의 외삼촌은 루이 13세로, 남편인 루이 14세는 외사촌 오빠가 된다.","{'word': '루이 14세', 'start_idx': 21, 'end_idx':...","{'word': '루이 13세', 'start_idx': 8, 'end_idx': ...",per:other_family,wikipedia


In [17]:
train

,id,sentence,subject_entity,object_entity,label,source
0,0,〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey R...,"{'word': '비틀즈', 'start_idx': 24, 'end_idx': 26...","{'word': '조지 해리슨', 'start_idx': 13, 'end_idx':...",no_relation,wikipedia
1,1,호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으...,"{'word': '민주평화당', 'start_idx': 19, 'end_idx': ...","{'word': '대안신당', 'start_idx': 14, 'end_idx': 1...",no_relation,wikitree
2,2,K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹으로부터...,"{'word': '광주FC', 'start_idx': 21, 'end_idx': 2...","{'word': '한국프로축구연맹', 'start_idx': 34, 'end_idx...",org:member_of,wikitree
3,3,균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪...,"{'word': '아성다이소', 'start_idx': 13, 'end_idx': ...","{'word': '박정부', 'start_idx': 22, 'end_idx': 24...",org:top_members/employees,wikitree
4,4,1967년 프로 야구 드래프트 1순위로 요미우리 자이언츠에게 입단하면서 등번호는 8...,"{'word': '요미우리 자이언츠', 'start_idx': 22, 'end_id...","{'word': '1967', 'start_idx': 0, 'end_idx': 3,...",no_relation,wikipedia
...,...,...,...,...,...,...
32465,32465,한국당은 7일 오전 9시부터 오후 5시까지 진행된 원내대표 및 정책위의장 후보자 등...,"{'word': '유기준', 'start_idx': 93, 'end_idx': 95...","{'word': '부산 서구·동구', 'start_idx': 100, 'end_id...",per:employee_of,wikitree
32466,32466,"법포는 다시 최시형, 서병학, 손병희 직계인 북접과 다시 서장옥, 전봉준, 김개남을...","{'word': '최시형', 'start_idx': 7, 'end_idx': 9, ...","{'word': '손병희', 'start_idx': 17, 'end_idx': 19...",per:colleagues,wikipedia
32467,32467,완도군(군수 신우철)이 국토교통부에서 실시한 '2019 교통문화지수 실태조사'에서 ...,"{'word': '완도군', 'start_idx': 0, 'end_idx': 2, ...","{'word': '신우철', 'start_idx': 7, 'end_idx': 9, ...",org:top_members/employees,wikitree
32468,32468,"중앙일보, JTBC 회장을 지낸 이후 중앙홀딩스 회장, 재단법인 한반도평화만들기 이...","{'word': 'JTBC', 'start_idx': 6, 'end_idx': 9,...","{'word': '중앙홀딩스', 'start_idx': 21, 'end_idx': ...",no_relation,wikipedia
